In [74]:
import pymysql
import pandas as pd
import os
from datetime import datetime

In [75]:
# 데이터베이스 연결 함수
def get_connection():
    return pymysql.connect(
        host=os.getenv('DB_HOST'),
        port=int(os.getenv('DB_PORT')),
        user=os.getenv('DB_USER'),
        password=os.getenv('DB_PASSWORD'),
        db=os.getenv('DB_NAME'),
        charset=os.getenv('DB_CHARSET'),
        cursorclass=pymysql.cursors.DictCursor
    )

In [76]:
def insert_article_data(df):  # new.csv를 받아 tbr_article 테이블에 INSERT 하는 함수
    # 데이터베이스 연결 설정
    connection = get_connection()
    try:
        with connection.cursor() as cursor:
            # 여러 값을 삽입하기 위한 SQL 쿼리 준비
            sql = """
            INSERT INTO tbr_article (platform_id, category_id, publication_date, update_date, title, content, publisher_id, article_url, author)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            # 삽입할 데이터 준비
            data = []
            for _, row in df.iterrows():
                platform_id = get_platform_id(row['platform'])
                category_id = get_category_id(row['platform'], row['category'])
                publisher_id = get_publisher_id(row['publisher'])
                publication_date = convert_date(row['publication_date'])
                update_date = convert_date(row['update_date']) if 'update_date' in row and pd.notna(row['update_date']) else None
                
                # NaN을 None으로 대체
                data.append((
                    platform_id, 
                    category_id, 
                    publication_date if pd.notna(publication_date) else None, 
                    update_date if pd.notna(update_date) else None, 
                    row['title'] if pd.notna(row['title']) else None, 
                    row['content'] if pd.notna(row['content']) else None, 
                    publisher_id if pd.notna(publisher_id) else None, 
                    row['article_url'] if pd.notna(row['article_url']) else None, 
                    row['editor'] if pd.notna(row['editor']) else None
                ))
            
            # 데이터와 함께 쿼리 실행
            cursor.executemany(sql, data)
        
        # 트랜잭션 커밋
        connection.commit()
        print("데이터가 tbr_article 테이블에 성공적으로 삽입되었습니다.")
    except Exception as e:
        print(f"오류 발생: {e}")
    finally:
        # 데이터베이스 연결 종료
        connection.close()

def get_platform_id(platform_name):
    # 플랫폼 이름을 ID로 매핑하는 로직 추가
    platform_mapping = {'다음': 2, '네이버': 1}  # 예시 매핑
    return platform_mapping.get(platform_name, None)

def get_category_id(platform, category_name):
    # 카테고리 매핑 정의
    category_mapping = {
        '다음': {
            '경제': 200,
            '금융': 201,
            # 기타 카테고리
        },
        '네이버': {
            '경제': 100,
            '금융': 101,
            # 기타 카테고리
        }
    }
    return category_mapping.get(platform, {}).get(category_name, None)

def get_publisher_id(publisher_name):
    # 출처 매핑 정의
    publisher_mapping = {
        '매경이코노미': 1,
        '경향신문': 2,
        '한국경제': 3,
        '파이낸셜뉴스': 4,
        # 추가 매핑
    }
    return publisher_mapping.get(publisher_name, None)

def convert_date(date_str):
    try:
        return datetime.strptime(date_str, '%Y. %m. %d. %H:%M') if pd.notna(date_str) else None
    except ValueError:
        return None

# 데이터프레임 로드
df = pd.read_csv('./csv/뉴스.csv')
sample_news = df[:500]

# 삽입 함수 실행
insert_article_data(sample_news)

데이터가 tbr_article 테이블에 성공적으로 삽입되었습니다.


In [77]:
def insert_user_data(df):  
    # 데이터베이스 연결 설정
    connection = get_connection()
    try:
        with connection.cursor() as cursor:
            # 여러 값을 삽입하기 위한 SQL 쿼리 준비
            sql = """
            INSERT INTO tbr_user (user_unique)
            VALUES (%s)
            """
            # 삽입할 데이터 준비
            data = []
            for _, row in df.iterrows():
                # NaN을 None으로 대체
                data.append((
                    row['user_unique'] if pd.notna(row['user_unique']) else None,
                ))
            
            # 데이터와 함께 쿼리 실행
            cursor.executemany(sql, data)
        
        # 트랜잭션 커밋
        connection.commit()
        print("데이터가 tbr_user 테이블에 성공적으로 삽입되었습니다.")
    
    except Exception as e:
        print(f"오류 발생: {e}")
    
    finally:
        # 데이터베이스 연결 종료
        connection.close()


# 데이터프레임 로드
df = pd.read_csv('./csv/댓글 수정.csv')
sample_user = df[:500]

# 삽입 함수 실행
insert_user_data(sample_user)

데이터가 tbr_user 테이블에 성공적으로 삽입되었습니다.


In [78]:
import pandas as pd
from datetime import datetime

def insert_commnet_data(comment, news):
    # 데이터베이스 연결 설정
    connection = get_connection()
    
    try:
        with connection.cursor() as cursor:
            # tbr_article 테이블에서 article_url과 id를 매핑하는 dict 생성
            cursor.execute("SELECT id, article_url FROM tbr_article")
            article_mapping = {row['article_url']: row['id'] for row in cursor.fetchall()}
            
            # tbr_user 테이블에서 user_unique와 id를 매핑하는 dict 생성
            cursor.execute("SELECT id, user_unique FROM tbr_user")
            user_mapping = {row['user_unique']: row['id'] for row in cursor.fetchall()}
            
            # 여러 값을 삽입하기 위한 SQL 쿼리 준비
            sql = """
            INSERT INTO tbr_comment (article_id, user_id, content, comment_date, update_date)
            VALUES (%s, %s, %s, %s, %s)
            """
            # 삽입할 데이터 준비
            data = []
            for _, row in comment.iterrows():
                # article_url을 article_id로 변환
                article_id = article_mapping.get(row['article_url'], None)
                
                # user_unique를 user_id로 변환
                user_id = user_mapping.get(row['user_unique'], None)
                
                comment_date = convert_date(row['comment_date'])
                update_date = convert_date(row['update_date']) if 'update_date' in row and pd.notna(row['update_date']) else None
                
                if article_id is not None and user_id is not None:
                    data.append((
                        article_id,
                        user_id,
                        row['comment'] if pd.notna(row['comment']) else None,
                        comment_date if pd.notna(comment_date) else None,
                        update_date if pd.notna(update_date) else None
                    ))
            
            # 데이터와 함께 쿼리 실행
            cursor.executemany(sql, data)
        
        # 트랜잭션 커밋
        connection.commit()
        print("데이터가 tbr_comment 테이블에 성공적으로 삽입되었습니다.")
    
    except Exception as e:
        print(f"오류 발생: {e}")
    
    finally:
        # 데이터베이스 연결 종료
        connection.close()

def convert_date(date_str):
    try:
        return datetime.strptime(date_str, '%Y. %m. %d. %H:%M') if pd.notna(date_str) else None
    except ValueError:
        return None

# 데이터프레임 로드
df = pd.read_csv('./csv/뉴스.csv')
sample_news = df[:500]
df = pd.read_csv('./csv/댓글 수정.csv')
sample_comment = df[:500]

# 삽입 함수 실행
insert_commnet_data(sample_comment, sample_news)


데이터가 tbr_comment 테이블에 성공적으로 삽입되었습니다.
